# Use Case: Using OpenAI API Summerize the web page content

### Import required libraries

In [1]:
# imports required libraries

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad to hear from you. How can I assist you today?


In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
client = Website("https://blog.stackademic.com/my-13-sources-of-income-as-a-software-engineer-1ffb7c69e4c8")
print(client.title)
print(client.text)

My 13 Sources of Income as a Software Engineer | by Mark Henry | Aug, 2025 | Stackademic
Sitemap
Open in app
Sign up
Sign in
Medium Logo
Write
Sign up
Sign in
Stackademic
·
Follow publication
Stackademic is a learning hub for programmers, devs, coders, and engineers. Our goal is to democratize free coding education for the world.
Follow publication
Member-only story
My 13 Sources of Income as a Software Engineer
Because One Paycheck Is Too Fragile to Rely On
Mark Henry
4 min read
·
5 days ago
--
12
Share
Let’s be honest:
no one becomes a software engineer just to stare at Jira tickets and debug some poor soul’s spaghetti code for 40 years.
We wanted the
freedom
, the flexibility, and —
let’s not lie — the money.
But here’s the kicker:
your cushy FAANG salary or decent mid-level paycheck isn’t bulletproof.
Layoffs happen. Tech bubbles burst. Your “rock-solid” job might evaporate faster than a free AWS credit.
I’ve been burned before.
One minute I was a “highly valued contributor,” the n

### Define Prompts for OpenAI API

In [7]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
# A function that writes a User Prompt that asks for summaries of websites:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
print(user_prompt_for(client))

You are looking at a website titled My 13 Sources of Income as a Software Engineer | by Mark Henry | Aug, 2025 | Stackademic
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Sitemap
Open in app
Sign up
Sign in
Medium Logo
Write
Sign up
Sign in
Stackademic
·
Follow publication
Stackademic is a learning hub for programmers, devs, coders, and engineers. Our goal is to democratize free coding education for the world.
Follow publication
Member-only story
My 13 Sources of Income as a Software Engineer
Because One Paycheck Is Too Fragile to Rely On
Mark Henry
4 min read
·
5 days ago
--
12
Share
Let’s be honest:
no one becomes a software engineer just to stare at Jira tickets and debug some poor soul’s spaghetti code for 40 years.
We wanted the
freedom
, the flexibility, and —
let’s not lie — the money.
But here’s the kicker:
your cushy FAANG salary or decent mid-level payche

### Prepare message for OpenAI API

In [10]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [11]:
messages_for(client)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled My 13 Sources of Income as a Software Engineer | by Mark Henry | Aug, 2025 | Stackademic\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nSitemap\nOpen in app\nSign up\nSign in\nMedium Logo\nWrite\nSign up\nSign in\nStackademic\n·\nFollow publication\nStackademic is a learning hub for programmers, devs, coders, and engineers. Our goal is to democratize free coding education for the world.\nFollow publication\nMember-only story\nMy 13 Sources of Income as a Software Engineer\nBecause One Paycheck Is Too Fragile to Rely On\nMark Henry\n4 min read\n·\n5 days ago\n--\n12\nShare\nLet’s be honest:\nno one becom

### Call OpenAI API to get the summary

In [12]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [13]:
## To see summary from Open AI in raw text format
summarize("https://vejr.tv2.dk/live/2024-12-02-vejr-og-klima")

"# Vejr og Klima Lige Nu - TV 2\n\n**Kort Oversigt:**\nTV 2's vejrsektion leverer opdateringer omkring aktuelle vejr- og klimaforhold i Danmark samt internationalt. Den dækker hændelser som kraftige byger, tordenvejr og hagl samt daglige vejrprognoser.\n\n## Seneste Opdateringer:\n- **Store Hagl:** Usædvanlige store hagl, målt til 2,5 cm, er faldet i Gåsetofte og Svenstrup ved Kalundborg. Dette er større end det typiske størrelse for hagl i Danmark.\n- **Tordenbyger:** Der er observeret truende rulleskyer over Slagelse, hvilket indikerer kraftige vindstød og tordenbyger i området.\n- **Generelt Vejr:** Lørdag forventes at starte med kraftige byger over Sjælland, som gradvist vil give plads til solskin senere på dagen. Vinden vil være let til jævn.\n- **Risiko for Skybrud:** TV 2 varsler risiko for skybrud i Nordsjælland fra fredag aften til lørdag morgen, hvilket kan føre til oversvømmelser.\n- **Vandtemperatur:** Vandtemperaturen omkring Danmark falder hurtigt, hvilket kan betyde, at 

In [14]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [15]:
display_summary("https://blog.stackademic.com/my-13-sources-of-income-as-a-software-engineer-1ffb7c69e4c8")

# Summary of "My 13 Sources of Income as a Software Engineer" by Mark Henry

In this article, Mark Henry discusses the importance of diversifying income streams for software engineers, asserting that relying on a single paycheck can be precarious due to potential job loss in an unstable tech environment. He shares his personal experiences with layoffs, noting that having multiple income sources provides financial security and peace of mind.

Henry outlines **13 different sources of income**, which vary in scale from significant amounts to smaller, supplementary incomes. The primary focus of the narrative is on the necessity and benefits of cultivating these diverse income streams, thereby avoiding the risks associated with dependence on a sole paycheck in the tech industry.

### Key Points
- **Full-Time Job**: Acknowledge the traditional income source.
- **Importance of Diversification**: Emphasizes building multiple income streams to mitigate financial risk.
- **Personal Experience**: Shares personal anecdotes about job insecurity and the impact of layoffs.

This piece is part of a broader goal within Stackademic to provide free coding education and resources for software professionals.